In [1]:
#@title Check GPU
!nvidia-smi

Thu Nov 24 00:00:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets --q

     |████████████████████████████████| 451 kB 7.8 MB/s 
     |████████████████████████████████| 212 kB 65.6 MB/s 
     |████████████████████████████████| 115 kB 86.0 MB/s 
     |████████████████████████████████| 182 kB 51.0 MB/s 
     |████████████████████████████████| 127 kB 83.5 MB/s 


In [3]:
!pip install transformers --quiet

     |████████████████████████████████| 5.5 MB 8.6 MB/s 
     |████████████████████████████████| 7.6 MB 74.4 MB/s 


In [4]:
#@title Imports
import nltk
import numpy as np
import tensorflow as tf
import pandas as pd
import transformers

from itertools import chain

from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from datasets import list_datasets, load_dataset
from pprint import pprint
from transformers import BertTokenizer, TFBertModel

from sklearn.model_selection import train_test_split


from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
import torch


In [7]:
#@title Load Dataset
imdb_test = load_dataset('imdb', split = 'test')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


In [8]:
!pip install transformers[sentencepiece] -q

     |████████████████████████████████| 1.3 MB 9.3 MB/s 


# Pegasus

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail"
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [11]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [12]:
def create_summary(dataset, model, tokenizer, 
                   batch_size=16, device=device, 
                   column_text="text"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    
    all_summaries = []
    
    for article_batch in article_batches:
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        
        #decode summary
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) 
        for s in summaries]

        all_summaries.append(decoded_summaries)
    
    
    return all_summaries

#### Create Summaries

In [13]:
len(imdb_test['label'])

25000

In [14]:
#@title Summary Creation
summary_test = create_summary(imdb_test, model_pegasus, tokenizer)

In [15]:
all_summaries = list(chain(*summary_test))

In [16]:
  df_pegasus = pd.DataFrame(all_summaries, columns = ['summary'])
  path_name = '/content/gdrive/MyDrive/imdb_test_pegasus_summary.csv'
  df_pegasus.to_csv(path_name)

# Pegasus Pretrained on Amazon Reviews

In [ ]:
!ls 

gdrive	sample_data


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = 'gdrive/MyDrive/model_weights/pegasus-amazon'
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def create_summary(dataset, model, tokenizer, 
                   batch_size=16, device=device, 
                   column_text="text"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    
    all_summaries = []
    
    for article_batch in article_batches:
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        
        #decode summary
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) 
        for s in summaries]

        all_summaries.append(decoded_summaries)
    
    
    return all_summaries

#### Create Summaries

In [ ]:
#@title Summary Creation
summary_test = create_summary(imdb_test, model_pegasus, tokenizer)

In [ ]:
all_summaries = list(chain(*summary_test))

In [ ]:
df_amazon = pd.DataFrame(all_summaries, columns = ['summary'])
path_name = '/content/gdrive/MyDrive/imdb_test_amazon_summary.csv'
df_amazon.to_csv(path_name)

# Finetuned Pegasus Pretrained on Amazon Reviews

In [ ]:
!ls 

gdrive	sample_data


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = 'gdrive/MyDrive/model_weights/pegasus-amazon2'
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def create_summary(dataset, model, tokenizer, 
                   batch_size=16, device=device, 
                   column_text="text"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    
    all_summaries = []
    
    for article_batch in article_batches:
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        
        #decode summary
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) 
        for s in summaries]

        all_summaries.append(decoded_summaries)
    
    
    return all_summaries

#### Create Summaries

In [ ]:
#@title Summary Creation
summary_test = create_summary(imdb_test, model_pegasus, tokenizer)

In [ ]:
all_summaries = list(chain(*summary_test))

In [ ]:
df_finetuned = pd.DataFrame(all_summaries, columns = ['summary'])
path_name = '/content/gdrive/MyDrive/imdb_test_amazon_finetuned_summary.csv'
df_finetuned.to_csv(path_name)